Users & Devices
===================================================

Cleaning up list of devices present at ITU for the entire period
We will look for:
- devices always connected to 1-3 devices in the entire period
- devices present for 'suspiciously long time


In [1]:
from lib.cached_data_loader import CachedDataLoader
from lib.panda_plotter import PandaPlotter
cdl = CachedDataLoader()
pp = PandaPlotter()


In [2]:
# COUNT DISTINCT DEVICES
sql = "SELECT COUNT(DISTINCT(wc.client_mac)) AS DEVICE_COUNT FROM wifi_clients AS wc;"
df = cdl.get_dataframe(sql)
print(df["DEVICE_COUNT"])

0    9127
Name: DEVICE_COUNT, dtype: int64


In [3]:
# COUNT DISTINCT USERS
sql = "SELECT COUNT(DISTINCT(wc.user_name)) AS USER_COUNT FROM wifi_clients AS wc;"
df = cdl.get_dataframe(sql)
print(df["USER_COUNT"])

0    7283
Name: USER_COUNT, dtype: int64


In [4]:
# DEVICES CONNECTED TO MAX 3 Access Points (stored as: v_wifi_clients_students_2)
# The following devices connect to only max 3 access points throughout the entire period
# These account for one third of the total number of devices 
# We can pretty much assume that these non-moving devices are not connected to students and
# therefore they are not useful for our purposes.
# Assumption: we assume that client_mac is a good way to identify devices(no changes)
sql = "SELECT wc.client_mac, " \
      "COUNT(DISTINCT(wc.device_id)) AS DEVCNT, " \
      "COUNT(DISTINCT(wc.mqtt_batch)) AS BATCHES " \
      "FROM wifi_clients AS wc " \
      "GROUP BY wc.client_mac " \
      "HAVING COUNT(DISTINCT(wc.device_id)) <= 3 " \
      "ORDER BY COUNT(DISTINCT(wc.mqtt_batch)) DESC"
df = cdl.get_dataframe(sql)
print(df)



                            client_mac  DEVCNT  BATCHES
0     NnA3K1RKTGxvK3dhVm5abFBBa2J4dz09       2    13155
1     ejdiWGpPdExMaGdWZHF5eDIvVHd1dz09       3    13153
2     cWdsWnpsazNsV1JPZHEzanFOemorZz09       2    13133
3     RWdsNnByclZJOGdMWW40YUN2U3JRUT09       3    12678
4     TVJ0eGlYWTVENXlTWHVxOFlyYTJOUT09       2    11243
...                                ...     ...      ...
3782  WVppZVQ2T29oNitaMVFGT0lrRXB5QT09       1        1
3783  ZzFrM3BRUmY3L3ZiS3hjQ2ExZHRpZz09       1        1
3784  WWVlOW5kQnd1SUNGMVBNWUs1RWtVQT09       1        1
3785  ZzZEdlMxZnhNOStnSHk5d1g4aFk3Zz09       1        1
3786  WXZ0eGNrczgxMDMrbFhCSU1lbHRjZz09       1        1

[3787 rows x 3 columns]


The above query (v_wifi_clients_students_2) still returns many 
devices that connect through networks outside ITU++ or eduroam -
counts: 
- not ITU++ or eduroam: 1047 - out of which:
- 5te: 36
- ITU-guest: 917
- ITU-guest-test: 1
- ITU-VPN: 1
- ITU-VPN2: 0
- sensors: 104
SELECT COUNT(DISTINCT(wcs2.client_mac))
FROM v_wifi_clients_students_2 AS wcs2
WHERE ssid IN ('ITU-guest')
;

Also, this many devices are found connecting on a non-"Authenticated"
session: 1741

SELECT COUNT(DISTINCT(wcs2.client_mac))
FROM v_wifi_clients_students_2 AS wcs2
WHERE `user_profile` <> 'Authenticated'
;

so we will create a combined view based on: v_wifi_clients_students
and v_wifi_clients_students_2 which will exclude:
- non moving devices
- authenticated sessions (user_profile = 'Authenticated')
- on wifi networks students can connect to AND be identified (ssid=ITU++|eduroam)
new view name: v_wifi_clients_students_3

In [5]:
# COUNT DISTINCT DEVICES
sql = "SELECT COUNT(DISTINCT(wcs3.client_mac)) AS DEVICE_COUNT FROM v_wifi_clients_students_3 AS wcs3;"
df = cdl.get_dataframe(sql)
print(df["DEVICE_COUNT"])

0    4837
Name: DEVICE_COUNT, dtype: int64


In [6]:
# COUNT DISTINCT USERS
sql = "SELECT COUNT(DISTINCT(wcs3.user_name)) AS USER_COUNT FROM v_wifi_clients_students_3 AS wcs3;"
df = cdl.get_dataframe(sql)
print(df["USER_COUNT"])




0    3272
Name: USER_COUNT, dtype: int64
